<a href="https://colab.research.google.com/github/skevin-dev/NLP-FELLOWSHIP/blob/week1/Parsing_IGihe_Raw_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import requests

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Get all the available captures since jan-2022

In [3]:
links =  requests.get('http://archive.org/wayback/available?url=igihe.com&timestamp=20220123')

In [4]:
links.json()

{'url': 'igihe.com', 'archived_snapshots': {}, 'timestamp': '20220123'}

In [5]:
snaps = []
err = []
for month in range(10):
    for day in range(31):
        link = requests.get('http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}'.format(month, day))
        try:
            if link.json()['archived_snapshots']['closest']['available']:
                snaps.append(link.json()['archived_snapshots']['closest']['url'])
        except KeyError:
            err.append(link.json())
            break

In [6]:
snaps_r  = set(snaps)

In [7]:
len(snaps_r)

11

In [8]:
snap_r = list(snaps_r)

In [9]:
len(snap_r)

11

In [10]:
str(snap_r[0])

'http://web.archive.org/web/20220603005738/https://www.igihe.com/'

In [11]:
b = requests.get(url=str(snap_r[0]))

In [12]:

links_art = []

for lin in snaps_r:
  content = requests.get(url=lin).content
  soup =  BeautifulSoup(content, "html.parser")
  articles = soup.find_all('span', class_="homenews-title")
  links_art.append(articles)

In [13]:
links_art[0][0].find('a')['href']

'imyidagaduro/article/the-ben-yageze-i-kampala-ahisha-umukunzi-we-itangazamakuru'

In [14]:
list(snaps_r)[0]

'http://web.archive.org/web/20220603005738/https://www.igihe.com/'

In [15]:
#Actual link
wayback_link = []
href_link = []

#Loop all the 184 snapshots
for index, snapshot in enumerate(list(snaps_r)):
  #In each title assign it's snapshop prefix link
  for title in links_art[index]:
    wayback_link.append(snapshot)
    href_link.append(str(title.find('a')['href']))

In [16]:
len(href_link)

803

In [17]:
df = pd.DataFrame(list(zip(wayback_link, href_link)), columns=['Prefix', 'Title'])

In [18]:
df.describe()

,Prefix,Title
count,803,803
unique,11,642
top,http://web.archive.org/web/20220603005738/http...,abantu/kubaho/urukundo/article/kubana-mutarash...
freq,73,4


In [19]:
df.head(10)

,Prefix,Title
0,http://web.archive.org/web/20220603005738/http...,imyidagaduro/article/the-ben-yageze-i-kampala-...
1,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/minisitiri-w-intebe-y...
2,http://web.archive.org/web/20220603005738/http...,amakuru/article/can-2023-amavubi-yakuye-inota-...
3,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/intumwa-za-mozambique...
4,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/perezida-tshisekedi-y...
5,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/mu-2021-leta-yahombye...
6,http://web.archive.org/web/20220603005738/http...,amakuru/mu-mahanga/article/u-burusiya-bumaze-g...
7,http://web.archive.org/web/20220603005738/http...,imyemerere/muzika/article/ibitaravuzwe-kuri-ro...
8,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/rab-yahumurije-abatur...
9,http://web.archive.org/web/20220603005738/http...,amakuru/muri-afurika/article/eac-iri-hafi-guka...


In [20]:
df.drop_duplicates(subset=['Title'])

,Prefix,Title
0,http://web.archive.org/web/20220603005738/http...,imyidagaduro/article/the-ben-yageze-i-kampala-...
1,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/minisitiri-w-intebe-y...
2,http://web.archive.org/web/20220603005738/http...,amakuru/article/can-2023-amavubi-yakuye-inota-...
3,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/intumwa-za-mozambique...
4,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/perezida-tshisekedi-y...
...,...,...
797,http://web.archive.org/web/20220901235955/http...,imyidagaduro/article/madamu-jeannette-kagame-y...
799,http://web.archive.org/web/20220901235955/http...,ikoranabuhanga/internet/article/abanyarwanda-b...
800,http://web.archive.org/web/20220901235955/http...,ikoranabuhanga/article/mu-2035-nta-modoka-ikor...
801,http://web.archive.org/web/20220901235955/http...,abantu/success-stories/article/iduka-ry-umuhin...


In [22]:
content = requests.get(df['Prefix'][0]+df['Title'][0]).content
raw_content =  BeautifulSoup(content, 'html.parser').find_all('div', class_="fulltext margintop10")
raw_content

raw_content_ = (BeautifulSoup(str(raw_content)).find_all('p'))
Cnts = []
# for i in range(len(raw_content_)-1):
#   y = open(path+'/text_files/file'+str(i+1)+'.txt', 'w',  encoding='utf-8')


In [23]:
def get_article_text(link):
  content = requests.get(link).content
  raw_content =  BeautifulSoup(content, 'html.parser').find_all('div', class_="fulltext margintop10")
  raw_content

  raw_content_ = (BeautifulSoup(str(raw_content)).find_all('p'))
  Cnts = []
  for i in range(len(raw_content_)-1):
    Cnts.append((raw_content_[i+1]).text)

  return Cnts

  


In [24]:
r = []
sample_df = df.head(50)
for i in range(len(sample_df)):
  d = {}
  u = get_article_text(sample_df['Prefix'][i]+sample_df['Title'][i])
  d['Links'] = sample_df['Prefix'][i]
  d['Titles'] = sample_df['Title'][i]
  d['Contents'] = u
  r.append(d)
  

In [26]:
data = pd.DataFrame(r)

In [27]:
data.head()

,Links,Titles,Contents
0,http://web.archive.org/web/20220603005738/http...,imyidagaduro/article/the-ben-yageze-i-kampala-...,[Mu kiganiro cyihariye yahaye IGIHE akigera ku...
1,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/minisitiri-w-intebe-y...,[uri uyu wa Kane yafunguye ku mugaragaro Inama...
2,http://web.archive.org/web/20220603005738/http...,amakuru/article/can-2023-amavubi-yakuye-inota-...,[Wari umukino w’umunsi wa mbere w’amatsinda yo...
3,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/intumwa-za-mozambique...,"[Mu ruzinduko rwabo mu Rwanda, iri tsinda riga..."
4,http://web.archive.org/web/20220603005738/http...,amakuru/u-rwanda/article/perezida-tshisekedi-y...,[Abasirikare birukanywe barimo Lt Col Kibibi M...
